In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
import optuna
import os
import numpy as np

## load data

In [13]:
country = 'afghan'
mode = 'Train'
feature = 'indices'

path = '/app/stella/dev/GeoITU/data'

X_indices = np.load(os.path.join(path, '{}_{}_{}.npy'.format(country, mode, feature)), allow_pickle=True)
y_labels = np.load(os.path.join(path, '{}_{}_labels.npy'.format(country, mode)), allow_pickle=True)

            
print(X_indices.shape, y_labels.shape)
assert X_indices.shape[0] == y_labels.shape[0]

X_indices = X_indices[:,:12].reshape(X_indices.shape[0], -1)
print(X_indices.shape)

(500, 18, 15) (500,)
(500, 180)


## rf

In [19]:
# Setting up the stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []  # List to store accuracy of each fold
f1_scores = []
fold_num = 0
num_trial = 50

# 5-fold Stratified Cross Validation loop
for train_index, test_index in skf.split(X_indices, y_labels):
    fold_num += 1

    # Splitting the dataset for this fold
    X_train, X_test = [X_indices[i] for i in train_index], [X_indices[i] for i in test_index]
    y_train_labels, y_test_labels = [y_labels[i] for i in train_index], [
        y_labels[i] for i in test_index
    ]
    
    
    # optimize params for single fold
    if fold_num == 1:  


        def objective(trial):
            params = {
            'n_estimators' : trial.suggest_int('n_estimators', 100, 1000),
            'max_depth' : trial.suggest_int('max_depth', 3, 15),
            # 'max_features' : trial.suggest_categorical('max_features', ['auto', 'sqrt']), 
            'min_samples_split' : trial.suggest_int('min_samples_split', 5, 32),
            'bootstrap' : trial.suggest_categorical('bootstrap', [True, False]),
            'n_jobs' : trial.suggest_categorical('n_jobs', [-1]) #fixed. use all cpus
            }

            clf = RandomForestClassifier(**params)
            clf.fit(X_train, y_train_labels)

            # Making predictions on the test set
            y_pred = clf.predict(X_test)

            # Calculating and reporting the accuracy
            accuracy = accuracy_score(y_test_labels, y_pred)
            return accuracy


        # optimize study
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=num_trial) 
        print(study.best_params)
                        
    # Model training
    clf = RandomForestClassifier(**study.best_params)  # change classifier here
    clf.fit(X_train, y_train_labels)
    
    # clf = xgb.XGBClassifier()
    # clf.fit(X_train, le.transform(y_train_labels))

    # Making predictions on the test set
    y_pred = clf.predict(X_test)

    # Calculating and reporting the accuracy
    accuracy = accuracy_score(y_test_labels, y_pred)
    accuracies.append(accuracy)  # Storing the accuracy
    
    # Calculating and reporting the fscore
    f_score = f1_score(y_test_labels, y_pred, average='weighted')
    f_scores.append(f_score)  # Storing the accuracy
    print(f"Fold {fold_num} Accuracy: {accuracy}")
    print(f"Fold {fold_num} F_score: {f_score}")

# Reporting the final results
avg_accuracy = np.mean(accuracies)
avg_fscore = np.mean(f_scores)
print(f"Average Accuracy across all folds: {avg_accuracy:.4f}")
print(f"Average Fscore across all folds: {avg_fscore:.4f}")


[I 2023-10-05 20:00:39,858] A new study created in memory with name: no-name-1677838a-786b-467e-99d3-d1fc22556b93
[I 2023-10-05 20:00:41,450] Trial 0 finished with value: 0.76 and parameters: {'n_estimators': 491, 'max_depth': 4, 'min_samples_split': 18, 'bootstrap': True, 'n_jobs': -1}. Best is trial 0 with value: 0.76.
[I 2023-10-05 20:00:41,795] Trial 1 finished with value: 0.75 and parameters: {'n_estimators': 149, 'max_depth': 3, 'min_samples_split': 15, 'bootstrap': False, 'n_jobs': -1}. Best is trial 0 with value: 0.76.
[I 2023-10-05 20:00:43,544] Trial 2 finished with value: 0.76 and parameters: {'n_estimators': 748, 'max_depth': 4, 'min_samples_split': 5, 'bootstrap': False, 'n_jobs': -1}. Best is trial 0 with value: 0.76.
[I 2023-10-05 20:00:45,471] Trial 3 finished with value: 0.76 and parameters: {'n_estimators': 799, 'max_depth': 11, 'min_samples_split': 32, 'bootstrap': False, 'n_jobs': -1}. Best is trial 0 with value: 0.76.
[I 2023-10-05 20:00:46,546] Trial 4 finished wi

{'n_estimators': 142, 'max_depth': 14, 'min_samples_split': 20, 'bootstrap': False, 'n_jobs': -1}
Fold 1 Accuracy: 0.77
Fold 1 F_score: 0.769792813532179
Fold 2 Accuracy: 0.86
Fold 2 F_score: 0.8599439775910364
Fold 3 Accuracy: 0.81
Fold 3 F_score: 0.8095238095238095
Fold 4 Accuracy: 0.88
Fold 4 F_score: 0.879951980792317
Fold 5 Accuracy: 0.78
Fold 5 F_score: 0.7785829307568438
Average Accuracy across all folds: 0.8200
Average Fscore across all folds: 0.8087


In [24]:
import catboost as ctb

ctb_clf = ctb.CatBoostClassifier(iterations=1700)

# Setting up the stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []  # List to store accuracy of each fold
f_scores = []
fold_num = 0
num_trial = 50

# 5-fold Stratified Cross Validation loop
for train_index, test_index in skf.split(X_indices, y_labels):
    fold_num += 1

    # Splitting the dataset for this fold
    X_train, X_test = [X_indices[i] for i in train_index], [X_indices[i] for i in test_index]
    y_train_labels, y_test_labels = [y_labels[i] for i in train_index], [
        y_labels[i] for i in test_index
    ]
    

    clf = ctb.CatBoostClassifier(iterations=2000)
    clf.fit(X_train, y_train_labels, verbose=False)

    # Making predictions on the test set
    y_pred = clf.predict(X_test)

    # Calculating and reporting the accuracy
    accuracy = accuracy_score(y_test_labels, y_pred)
    accuracies.append(accuracy)  # Storing the accuracy
    
    # Calculating and reporting the fscore
    f_score = f1_score(y_test_labels, y_pred, average='weighted')
    f_scores.append(f_score)  # Storing the accuracy
    print(f"Fold {fold_num} Accuracy: {accuracy}")
    print(f"Fold {fold_num} F_score: {f_score}")

# Reporting the final results
avg_accuracy = np.mean(accuracies)
avg_fscore = np.mean(f_scores)
print(f"Average Accuracy across all folds: {avg_accuracy:.4f}")
print(f"Average Fscore across all folds: {avg_fscore:.4f}")


Fold 1 Accuracy: 0.78
Fold 1 F_score: 0.7799119647859143
Fold 2 Accuracy: 0.87
Fold 2 F_score: 0.8698828946051447
Fold 3 Accuracy: 0.82
Fold 3 F_score: 0.8193496587715776
Fold 4 Accuracy: 0.89
Fold 4 F_score: 0.88998899889989
Fold 5 Accuracy: 0.79
Fold 5 F_score: 0.7882851093860268
Average Accuracy across all folds: 0.8300
Average Fscore across all folds: 0.8295
